# Convolutional Neural Network

### Importing the libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.19.0'

# Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,  #feature scaling to each and every value by diving all value 255 .
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory(r"D:\PROJECT\LIPS\DataSet\train",
                                                 target_size = (64,64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 color_mode = 'grayscale'  # Add this if images are grayscale
                                                )

Found 3798 images belonging to 3 classes.


### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(r"D:\PROJECT\LIPS\DataSet\test",
                                            target_size = (64, 64),
                                            batch_size = 32,  #how many images in each batch
                                            class_mode = 'categorical',
                                           color_mode = 'grayscale'  # Add this if images are grayscale
                                           )

Found 951 images belonging to 3 classes.


# Part 2 - Building the CNN

### Adding the first convolutional layer

### Step 1 - Convolution

In [ ]:
cnn = tf.keras.models.Sequential([tf.keras.layers.Input(shape=(64, 64, 3)), tf.keras.layers.Conv2D(32, (3, 3), activation='relu')])

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
#rectifier activation 

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=3, activation='sigmoid'))

# Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss ='categorical_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
from tensorflow.keras.utils import PyDataset

class CustomDataset(PyDataset):
    def __init__(self, data, **kwargs):
        super().__init__(**kwargs)
        self.data = data

# Part 4 - Making a single prediction

In [ ]:
cnn.build(input_shape=(None, 64, 64, 3))

In [ ]:
cnn.save('D:\PROJECT\LIPS\model.keras')

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img(r"D:\PROJECT\LIPS\DataSet\test\PNEUMONIA\005.jpeg", target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step


{'HEALTHY': 0, 'PNEUMONIA': 1, 'TUBERCULOSIS': 2}

In [ ]:
# print(np.argmax(result[0]))

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the trained model
from tensorflow.keras.models import load_model
cnn = load_model('model.keras')

# Define the class labels
class_labels = {0: 'HEALTHY', 1: 'PNEUMONIA', 2: 'TUBERCULOSIS'}

# Function to make predictions
@tf.function
def predict(image_path):
    test_image = image.load_img(image_path, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = cnn(test_image)  # Call the model directly on the input
    label_index = tf.argmax(result, axis=1)  # Find the index of the maximum value
    return label_index

# Example usage
image_path = r"D:\PROJECT\LIPS\DataSet\test\TUBERCULOSIS\t006.jpg"
result = predict(image_path)
prediction_index = result.numpy()[0]
prediction_label = class_labels[prediction_index]
print("Prediction:", prediction_label)

D:\PROJECT\LIPS\my_venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Prediction: PNEUMONIA


In [ ]:
def predict(image_path):
    test_image = image.load_img(image_path, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = test_image / 255.0  # Normalize pixel values
    test_image = np.expand_dims(test_image, axis=0)
    result = cnn(test_image)
    label_index = tf.argmax(result, axis=-1)
    return label_index

In [ ]:
import matplotlib.pyplot as plt
test_image = image.load_img(r"D:\PROJECT\LIPS\DataSet\test\TUBERCULOSIS\048.jpeg", target_size=(64, 64))
plt.imshow(test_image)
plt.title("Input Image")
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\PROJECT\\LIPS\\DataSet\\test\\TUBERCULOSIS\\048.jpeg'

In [ ]:
cnn = load_model('model.keras', compile=False)
cnn.summary()  # Add this line

In [ ]:
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
base_model.trainable = False  # Freeze layers

cnn = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
cnn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
def predict(image_path):
    test_image = image.load_img(image_path, target_size=(64, 64))
    test_image = image.img_to_array(test_image) / 255.0
    test_image = np.expand_dims(test_image, axis=0)
    probabilities = cnn.predict(test_image)[0]
    predicted_class = np.argmax(probabilities)
    print(f"Probabilities: {probabilities}")
    return predicted_class

In [ ]:
# if np.argmax(result[0])==0:
#     print("HEALTHY")
# elif np.argmax(result[0])==1:
#     print("PNEUMONIA")
# else:
#     print("TUBERCULOSIS")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.plot(h.history['loss'], label='Training Loss')  
plt.plot(h.history['val_loss'], label='Validation Loss')  
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend() 
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
pred = np.argmax(cnn.predict(test_set),axis=1)

In [ ]:
cm = confusion_matrix(np.argmax(test_set,axis=1),pred)
print(cm)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(cm,annot=True)
plt.show()